## Problem 1

The Nernst equation (equation 5.4) was derived in chapter 5 under the assumption that the membrane potential was negative and the ion being considered was positively charged. Rederive this result for a negatively charged ion and for the case when E is positive to verify that it applies in all these cases.

$$E=\frac{V_T}{z}\ln\left(\frac{[\text{outside}]}{[\text{inside}]}\right)$$

### Solution

TODO

## Problem 2

Verify that equation 5.47

$$V(t)=V_\infty+(V(t_0)-V_\infty)\exp(-(t-t_0)/\tau_V)$$

is a solution of equation 5.46

$$\tau_V\frac{dV}{dt}=V_\infty-V$$

when $V_\infty$ is independent of time. Then, solve equation 5.46 for the case when $V_\infty$ is an arbitrary function of time. In this solution, $V(t)$ is expressed in terms of integrals involving $V_\infty(t)$.

### Solution

TODO

## Problem 3

Build a model integrate-and-fire neuron from equation 5.8.

$$\tau_m\frac{dV}{dt}=E_L-V+R_mI_e$$

Use $V_\text{rest} = -70$ mV, $R_m = 10 M\Omega$, and $\tau_m = 10$ ms. Initially set $V = V_\text{rest}$. When the membrane potential reaches $V_\text{th} = -54$ mV, make the neuron fire a spike and reset the potential to $V\text{reset} = -80$ mV. Show sample voltage traces (with spikes) for a 300-ms-long current pulse (choose a reasonable current $I_e$) centered in a 500-ms-long simulation. Determine the firing rate of the model for various magnitudes of constant $I_e$ and compare the results with equation 5.11.

$$r_\text{isi}=\frac{1}{t_\text{isi}}=\left(\tau_m\ln\left(\frac{R_m I_e + E_L -V_\text{reset}}{R_mI_e+E_L-V_\text{th}}\right)\right)^{-1}$$

### Solution

TODO

## Problem 4

Include an extra current in the integrate-and-fire model to introduce spike-rate adaptation, as described in equations 5.13

$$\tau_m\frac{dV}{dt}=E_L-V-r_mg_\text{sra}(V-E_K)+R_mI_e$$

and 5.14,

$$\tau_\text{sra}\frac{dg_\text{sra}}{dt}=-g_\text{sra}$$

and in the caption to figure 5.6.

### Solution

TODO

### Problem 5

Add an excitatory synaptic conductance to the integrate-and-fire neuron of exercise 3 by adding the extra synaptic conductance term in equation 5.43

$$\tau_m\frac{dV}{dt}=E_L-V-r_m\bar{g}_sP_s(V-E_s)+R_mI_e$$

with $E_s = 0$. Set the external current to zero, $I_e = 0$, in this example, and assume that the probability of release on receipt of a presynaptic spike is 1. Use $r_m\bar{g}_s = 0.5$ and describe $P_s$ using the alpha function of equation 5.35 with $\tau_s = 10$ ms and $P_\text{max} = 0.5$. To incorporate multiple presynaptic spikes, $P_s$ should be described by a pair of differential equations,

$$\tau_s\frac{dP_s}{dt}=eP_\text{max}z-P_s$$

with $e = \exp(1)$, and

$$\tau_s\frac{dz}{dt} = -z$$,

with the additional rule that $z$ is set to 1 whenever a presynaptic spike arrives. Plot V(t) in one graph and the synaptic current in another. Trigger synaptic events at times 50, 150, 190, 300, 320, 400, and 410 ms. Explain what you see.

### Solution

TODO

### Problem 6

The equations in exercise 5 generate an α function response to a single input spike, but they do not prevent P s from growing greater than 1 when the model synapse is driven by multiple spikes at a sufficiently high frequency. In other words, this model synapse does not saturate. To introduce saturation, modify the equations of exercise 5 to

$$\tau_s\frac{dP_s}{dt} = e P_\text{max}z(1 − P_s) − P_s$$

with $e = \exp(1)$, and

$$\tau_s\frac{dz}{dt} = -z$$,

with the additional rule that $z$ is set to 1 whenever a presynaptic spike arrives. Compare $P_s(t)$ computed using these equations with $P_s(t)$ computed using the equations of exercise 5 for constant rate, regular (periodic) presynaptic spike trains with frequencies ranging from 1 to 100 Hz. In both cases, use $\tau_s = 10$ ms and $P_\text{max} = 0.5$.

### Solution

TODO

## Problem 7

Construct a model of two coupled integrate-and-fire neurons similar to that of figure 5.20. Both model neurons obey equation 5.43

$$\tau_m\frac{dV}{dt}=E_L-V-r_m\bar{g}_sP_s(V-E_s)+R_mI_e$$

with $E_L = -70$ mV, $V_\text{th} = 54$ mV, $V_\text{reset} = -80$ mV, $\tau_m = 20$ ms, $r_m g_s = 0.15$, and $R_mI e = 18$ mV. Both synapses should be described as in exercise 5 with $P_\text{max} = 0.5$ and $\tau_s = 10$ ms. Consider cases where both synapses are excitatory, with $E_s = 0$ mV, and both are inhibitory, with $E_s = -80$ mV. Show how the pattern of firing for the two neurons depends on the type (excitatory or inhibitory) of the reciprocal synaptic connections. For these simulations, set the initial membrane voltages of the two neurons to slightly different values, randomly, and run the simulation until an equilibrium situation has been reached, which may take a few seconds of simulated run time. Start from a few different random initial conditions to study whether the results are consistent. Investigate what happens if you change the strengths and time constants of the reciprocal synapses.

### Solution

TODO

## Problem 8

Build a Hodgkin-Huxley model neuron by numerically integrating the equations for $V$, $m$, $h$, and $n$ given in chapter 5 (see, in particular equations 5.6,

$$c_m\frac{dV}{dt}=-i_m+\frac{I_e}{A}$$

5.17–5.19,

$$\tau_n(V)\frac{dn}{dt}=n_\infty(V)-n$$
$$\tau_n(V)=\frac{1}{\alpha_n(V)+\beta_n(V)}$$
$$n_\infty(V)=\frac{\alpha_n(V)}{\alpha_n(V)+\beta_n(V)}$$

5.22,
$$\alpha_n=\frac{0.01(V+55)}{1-\exp(-0.1(V+55))}\,\text{ and }\,\beta_n=0.125\exp(-0.0125(V+65))$$

5.24,
$$\alpha_m=\frac{0.1(V+40)}{1-\exp(-0.1(V+40))}\,\text{ }\,\beta_m=4\exp(-0.0556(V+65))$$
$$\alpha_h=0.07\exp(-0.05(V+65))\,\text{ }\,\beta_h=1/(1+\exp(-0.1(V+35)))$$

and 5.25).
$$i_m=\bar{g}_L(V-E_L)+\bar{g}_Kn^4(V-E_K)+\bar{g}_\text{Na}m^3h(V-E_\text{Na})$$


Take $c_m = 10$ nF/mm$^2$ , and as initial values take: $V = -65$ mV, $m = 0.0529$, $h = 0.5961$, and $n = 0.3177$. Use an integration time step of 0.1 ms. Use an external current with $I_e/A = 200$ nA/mm$^2$ and plot $V$, $m$, $h$, and $n$ as functions of time for a suitable interval. Also, plot the firing rate of the model as a function of $I_e/A$ over the range from 0 to 500 nA/mm$^2$ . Show that the firing rate jumps discontinuously from zero to a finite value when the current passes through the minimum value required to produce sustained firing. Finally, apply a pulse of negative current with $I_e/A = -50$ nA/mm$^2$ for 5 ms followed by $I_e/A = 0$ and show what happens.

### Solution

TODO

## Problem 9

Construct and simulate the K$^+$ channel model of figure 5.12. Plot the mean squared deviation between the current produced by $N$ such model channels and the Hodgkin-Huxley current as a function of $N$ over the range $1 \le N \le 100$, matching the amplitude of the Hodgkin-Huxley model so that the mean currents are the same.

### Solution

TODO

## Problem 10

Compute analytically the value of the release probability $P_\text{rel}$ just before the time of each presynaptic spike for a regular (periodic rather than Poisson), constant-frequency presynaptic spike train as a function of the presynaptic firing rate. Do this for both the depression and facilitation models described by equation 5.37.

$$\tau_P\frac{dP_\text{rel}}{dt}=P_0-P_\text{rel}$$

### Solution

TODO